In [1]:
import pandas as pd
from urllib import request
import os
from shutil import rmtree
import re
import __main__ as main 
main.__file__ = "main_file"
from spyre import server

In [2]:
#створення дф, підготовка його
def create_dataframe(csv_path, p_id):
    headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty']
    df = pd.read_csv(csv_path, header=1, names=headers)
    df = df[df['VHI'] != -1]
    df.at[0, 'Year'] = df.at[0, 'Year'][9:]
    df = df.drop(columns='empty').drop(df.index[-1])
    df['area'] = p_id
    df['Year'] = df['Year'].astype(int)
    return df

In [3]:
#Збирає всі шляхи до файлів в папці дата
def paths_list():
    file_paths = []
    for root, dirs, files in os.walk('data'):
        for i in files:
            file_paths.append(os.path.join(root, i))

    return file_paths


In [4]:

#Вилучає ідентифікатор провінції
def get_p_id(file_path):
    with open(file_path, 'r') as file:
        first_line = file.readline()
        match = re.search(r'Province\s*=\s*(\d+)', first_line)
        return int(match.group(1))


#Заміна індексів
provinces_dict = {1: 22, 2: 24, 3: 23, 4: 25, 5: 3, 6: 4,
                  7: 8, 8: 19, 9: 20, 10: 21, 11: 9, 12: 9,
                  13: 10, 14: 11, 15: 12, 16: 13, 17: 14,
                  18: 15, 19: 16, 20: 25, 21: 17, 22: 18,
                  23: 6, 24: 1, 25: 2, 26: 7, 27: 5}

#Створення дф і об'єднання їх у мейн
main_df = pd.DataFrame()
for file_path in paths_list():
    p_id = provinces_dict[get_p_id(file_path)]
    df = create_dataframe(file_path, p_id)
    main_df = pd.concat([main_df, df], ignore_index=True)

In [5]:
class SimpleApp(server.App):
    title = "NOAA data visualization"
    inputs = [
            { "type": 'dropdown',
                    "label": "NOAA data dropdown",
                    "options" : [ {"label": "VCI", "value":"VCI" },
                        {"label":"TCI", "value":"TCI"},
                        {"label":"VHI", "value":"VHI"}],
                    "key": 'ticker',
                    'value':'VHI',
                    "action_id": "update_data"
            },
            {
                        "type": 'dropdown',
                        "label": "NOAA ",
                        "options" : [ {"label": "1: Cherkasy", "value":1 },
                            {"label":"2:Chernihiv", "value":2},
                            {"label":"3:Chernivtsi", "value":3},
                            {"label":"4:Crimea", "value":"4:Crimea"},
                            {"label":"5:Dnipropetrovs`k", "value":5},
                            {"label":"6:Donets`k", "value":6},
                            {"label":"7:Ivano-Frankivsk", "value":7},
                            {"label":"8:Kharkiv", "value":8},
                            {"label":"9:Kherson", "value":9},
                            {"label":"10:Khmelnytskyy", "value":10},
                            {"label":"11:Kiev", "value":11},
                            {"label":"12:Kiev City", "value":12},
                            {"label":"13:Kirovohrad", "value":13},
                            {"label":"14:Luhansk", "value":14},
                            {"label":"15:L`viv", "value":15},
                            {"label":"16:Mykolaiv", "value":16},
                            {"label":"17:Odessa", "value":17},
                            {"label":"18:Poltava", "value":18},
                            {"label":"19:Rivne", "value":19},
                            {"label":"20:Sevastopol`", "value":20},
                            {"label":"21:Sumy", "value":21},
                            {"label":"22:Ternopil", "value":22},
                            {"label":"23:Transcarpathia", "value":23},
                            {"label":"24:Vinnytsya", "value":24},
                            {"label":"25:Volyn", "value":25},
                            {"label":"26:Zaporizhzhya", "value":26},
                    ],
                        "key": 'area',
                        'value':6,
                        "action_id": "update_data" 
            },{
                            "type": 'slider',
                            "label": 'Indicate the year of the beginning of the interval',
                            "min": 1982, "max": 2024,
                            "value": 1990,
                            "key": 'year_first',
                            "action_id": 'update_data'
            },{
                            "type": 'slider',
                            "label": 'Indicate the year of the end  of the interval',
                            "min": 1982, "max": 2024,
                            "value": 2000,
                            "key": 'year_second',
                            "action_id": 'update_data'
            },{
                            "type": "text",
                            "label": "Specify the weekly interval",
                            "action_id": "update_data",
                            "value": "1,2",
                            "key":"week"
            } 
                
        ]
    
    controls = [{   "control_type" : "hidden",  
                    "label" : "NOAA",
                    "control_id" : "update_data"}]
    
    tabs = ["Plot", "Table"]

    outputs = [
        {
            "type": "plot",
            "id": "plot",
            "control_id": "update_data",
            "tab": "Plot"},
        {
            "type": "table",
            "id": "table_id",
            "control_id": "update_data",
            "tab": "Table",
            "on_page_load": True
        }
    ]

    # df.head()
    
    def getData(self, params):
        
        ticker = params['ticker']
        area = int(params['area'])
        year_first = int(params['year_first'])
        year_second = int(params['year_second'])

        week = (params['week'].split(','))
        week_start = int(week[0].strip())  
        week_end = int(week[1].strip())

        final_df = main_df[(main_df['area'] == area) & 
                           (main_df['Year'] >= year_first) & (main_df['Year'] <= year_second) &
                          (main_df['Week'] >= week_start) & (main_df['Week'] <= week_end)]

        return final_df[['Year', 'Week',ticker]]
    

    def getPlot(self, params):
        df_f = self.getData(params).set_index(['Year', 'Week'])
        obj = df_f.plot(figsize=(15, 6))
        fig = obj.get_figure()
        
        return fig

In [ ]:
app = SimpleApp()
app.launch()

[16/Nov/2024:14:31:50] ENGINE Listening for SIGTERM.
INFO:cherrypy.error:[16/Nov/2024:14:31:50] ENGINE Listening for SIGTERM.
[16/Nov/2024:14:31:50] ENGINE Bus STARTING
INFO:cherrypy.error:[16/Nov/2024:14:31:50] ENGINE Bus STARTING
[16/Nov/2024:14:31:50] ENGINE Set handler for console events.
INFO:cherrypy.error:[16/Nov/2024:14:31:50] ENGINE Set handler for console events.
CherryPy Checker:
The Application mounted at '' has an empty config.

[16/Nov/2024:14:31:50] ENGINE Started monitor thread 'Autoreloader'.
INFO:cherrypy.error:[16/Nov/2024:14:31:50] ENGINE Started monitor thread 'Autoreloader'.
[16/Nov/2024:14:31:51] ENGINE Serving on http://127.0.0.1:8080
INFO:cherrypy.error:[16/Nov/2024:14:31:51] ENGINE Serving on http://127.0.0.1:8080
[16/Nov/2024:14:31:51] ENGINE Bus STARTED
INFO:cherrypy.error:[16/Nov/2024:14:31:51] ENGINE Bus STARTED
